# TensorFlow Assignment: Multilayer Perceptron (MLP) and Convolutional Neural Network (CNN)

**[Duke Community Standard](http://integrity.duke.edu/standard.html): By typing your name below, you are certifying that you have adhered to the Duke Community Standard in completing this assignment.**

Name: [Siyuan, Liu]

Now that you've run through a simple logistic regression model on MNIST, let's see if we can do better (Hint: we can). For this assignment, you'll build a multilayer perceptron (MLP) and a convolutional neural network (CNN), two popular types of neural networks, and compare their performance. Some potentially useful code:

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# Import data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
# Helper functions for creating weight variables
def weight_variable(shape):
    """weight_variable generates a weight variable of a given shape."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    """bias_variable generates a bias variable of a given shape."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# Tensorflow Functions that might also be of interest:
# tf.nn.sigmoid()
# tf.nn.relu()

### Multilayer Perceptron

Build a multilayer perceptron for MNIST digit classfication. Feel free to play around with the model architecture and see how the training time/performance changes, but to begin, try the following:

Image -> fully connected (500 hidden units) -> nonlinearity (Sigmoid/ReLU) -> fully connected (10 hidden units) -> softmax

Skeleton framework for you to fill in (Code you need to provide is marked by `###`):

In [14]:
# Model Inputs
x = tf.placeholder(tf.float32, [None, 784])### MNIST images enter graph here ###
y_ = tf.placeholder(tf.float32, [None, 10])### MNIST labels enter graph here ###

# Define the graph
W_h1 = weight_variable([784, 500])
b_h1 = bias_variable([500])
b_h2 = bias_variable([10])

h1 = tf.nn.relu(tf.matmul(x, W_h1) + b_h1)
W_out = weight_variable([500, 10])
y_out = tf.matmul(h1, W_out) + b_h2
y_mlp = tf.nn.softmax(y_out)

### Create your MLP here##
### Make sure to name your MLP output as y_mlp ###



# Loss 
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_mlp))

# Optimizer
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

# Evaluation
correct_prediction = tf.equal(tf.argmax(y_mlp, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

import time
with tf.Session() as sess:
    # Initialize all variables
    sess.run(tf.global_variables_initializer())
    
    start_time = time.time()
    # Training regimen
    for i in range(4000):
        # Validate every 250th batch
        if i % 250 == 0:
            validation_accuracy = 0
            for v in range(10):
                batch = mnist.validation.next_batch(100)
                validation_accuracy += (1/10) * accuracy.eval(feed_dict={x: batch[0], y_: batch[1]})
            print('step %d, validation accuracy %g' % (i, validation_accuracy))
        
        # Train    
        batch = mnist.train.next_batch(50)
        train_step.run(feed_dict={x: batch[0], y_: batch[1]})
    print('training time is %s' % (time.time() - start_time))
    print('test accuracy %g' % accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

step 0, validation accuracy 0.114
step 250, validation accuracy 0.773
step 500, validation accuracy 0.844
step 750, validation accuracy 0.839
step 1000, validation accuracy 0.925
step 1250, validation accuracy 0.951
step 1500, validation accuracy 0.936
step 1750, validation accuracy 0.955
step 2000, validation accuracy 0.97
step 2250, validation accuracy 0.959
step 2500, validation accuracy 0.958
step 2750, validation accuracy 0.958
step 3000, validation accuracy 0.97
step 3250, validation accuracy 0.961
step 3500, validation accuracy 0.965
step 3750, validation accuracy 0.961
training time is 28.21126437187195
test accuracy 0.9652


#### Comparison

How do the sigmoid and rectified linear unit (ReLU) compare?

***

As for the performance of sigmoid and ReLU in this MLP structure, the performance of ReLU is better than that of sigmoid function. The accuracy of MLP using ReLU on MNIST can reach to 96%, however, using sigmoid only get an accuracy around 84%.

As for the training time, using ReLU is slightly faster than using sigmoid. On my machine, it costs 25.0 seconds to complete the training phase if ReLU is used, and costs 25.7 seconds if sigmoid is applied.
***

### Convolutional Neural Network

Build a simple 2-layer CNN for MNIST digit classfication. Feel free to play around with the model architecture and see how the training time/performance changes, but to begin, try the following:

Image -> CNN (32 5x5 filters) -> nonlinearity (ReLU) ->  (2x2 max pool) -> CNN (64 5x5 filters) -> nonlinearity (ReLU) -> (2x2 max pool) -> fully connected (1024 hidden units) -> nonlinearity (ReLU) -> fully connected (10 hidden units) -> softmax

Some additional functions that you might find helpful:

In [13]:
# Convolutional neural network functions
def conv2d(x, W):
    """conv2d returns a 2d convolution layer with full stride."""
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    """max_pool_2x2 downsamples a feature map by 2X."""
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# Tensorflow Function that might also be of interest:
# tf.reshape()

Skeleton framework for you to fill in (Code you need to provide is marked by `###`):

*Hint: Convolutional Neural Networks are spatial models. You'll want to transform the flattened MNIST vectors into images for the CNN. Similarly, you might want to flatten it again sometime before you do a softmax. You also might want to look into the  [conv2d() documentation](https://www.tensorflow.org/api_docs/python/tf/nn/conv2d) to see what shape kernel/filter it's expecting.*

In [15]:
# Model Inputs
x = tf.placeholder(tf.float32, [None, 784])### MNIST images enter graph here ###
y_ = tf.placeholder(tf.float32, [None, 10])### MNIST labels enter graph here ###

# Define the graph
x_2d = tf.reshape(x,[-1,28,28,1])
#[filter_height, filter_width, in_channels, out_channels]
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
conv1_out = conv2d(x_2d,W_conv1) + b_conv1
relu1_out = tf.nn.relu(conv1_out)
pool1_out = max_pool_2x2(relu1_out)
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])
conv2_out =conv2d(pool1_out,W_conv2) + b_conv2
relu2_out = tf.nn.relu(conv2_out)
pool2_out = max_pool_2x2(relu2_out)
#flatten for the FC layer
fc1_in = tf.reshape(pool2_out,[-1,64*49])
W_fc1 = weight_variable([64*49,1024])
b_fc1 = bias_variable([1024])
fc1_out = tf.nn.relu(tf.matmul(fc1_in,W_fc1) + b_fc1)
W_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])
fc2_out = tf.matmul(fc1_out,W_fc2) + b_fc2
y_conv = tf.nn.softmax(fc2_out)

### Create your CNN here##
### Make sure to name your CNN output as y_conv ###



# Loss 
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))

# Optimizer
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# Evaluation
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    # Initialize all variables
    sess.run(tf.global_variables_initializer())
    
    # Training regimen
    for i in range(10000):
        # Validate every 250th batch
        if i % 250 == 0:
            validation_accuracy = 0
            for v in range(10):
                batch = mnist.validation.next_batch(50)
                validation_accuracy += (1/10) * accuracy.eval(feed_dict={x: batch[0], y_: batch[1]})
            print('step %d, validation accuracy %g' % (i, validation_accuracy))
        
        # Train    
        batch = mnist.train.next_batch(50)
        train_step.run(feed_dict={x: batch[0], y_: batch[1]})

    print('test accuracy %g' % accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

step 0, validation accuracy 0.08
step 250, validation accuracy 0.718
step 500, validation accuracy 0.854
step 750, validation accuracy 0.85
step 1000, validation accuracy 0.964
step 1250, validation accuracy 0.962
step 1500, validation accuracy 0.97
step 1750, validation accuracy 0.964
step 2000, validation accuracy 0.966
step 2250, validation accuracy 0.982
step 2500, validation accuracy 0.988
step 2750, validation accuracy 0.986
step 3000, validation accuracy 0.986
step 3250, validation accuracy 0.986
step 3500, validation accuracy 0.98
step 3750, validation accuracy 0.982
step 4000, validation accuracy 0.986
step 4250, validation accuracy 0.988
step 4500, validation accuracy 0.99
step 4750, validation accuracy 0.982
step 5000, validation accuracy 0.988
step 5250, validation accuracy 0.982
step 5500, validation accuracy 0.986
step 5750, validation accuracy 0.988
step 6000, validation accuracy 0.974
step 6250, validation accuracy 0.99
step 6500, validation accuracy 0.994
step 6750, va

Some differences from the logistic regression model to note:

- The CNN model might take a while to train. Depending on your machine, you might expect this to take up to half an hour. If you see your validation performance start to plateau, you can kill the training.

- The logistic regression model we used previously was pretty basic, and as such, we were able to get away with using the GradientDescentOptimizer, which performs implements the gradient descent algorithm. For more difficult optimization spaces (such as the ones deep networks pose), we might want to use more sophisticated algorithms. Prof David Carlson has a lecture on this later.
    
- Because of the larger size of our network, notice that our minibatch size has shrunk.
    
- We've added a validation step every 250 minibatches. This let's us see how our model is doing during the training process, rather than sit around twiddling our thumbs and hoping for the best when training finishes. This becomes especially significant as training regimens start approaching days and weeks in length. Normally, we validate on the entire validation set, but for the sake of time we'll just stick to 10 validation minibatches (500 images) for this homework assignment.

#### Comparison

How do the MLP and CNN compare in accuracy? Training time? Why would you use one vs the other? Is there a problem you see with MLPs when applied to other image datasets?

***

As for accuracy, CNNs has a higher accuracy than MLPs. For MNIST, the accuracy of CNNs is over 99%, that of MLPs(using ReLU as activation function) can achieve an accuracy of 96%. However, the training phase of CNNs is much slower than MLP. 

For a small and simple image dataset, I would prefer using MNIST to train and classify the images, due to the high computational cost of CNNs and the accuracy of MLPs is only slightly less than that of CNNs. 

For a magnificent dataset, like ImageNet database. I would like to choose CNNs to do the classification. If using MLPs to do the task, the net can be very large and there will be too much parameters to learn, which may lead to overfitting. The convolutional layers in the CNNs can use less parameters to extract the features in the images, making the training process effeciently and easier to achieve high accuracy 
***